In [0]:
# PARAMETERS:

catalog = 'dev'

# Schemas
bronze_db = 'db1_bronze_raw'
silver_db = 'db1_silver'
gold_db = 'db1_gold'

# Table
trans_tbl = 'transactions'

cleaned_tbl = 'clean_transactions'

In [0]:

from pyspark.sql import SparkSession

from pyspark.sql.functions import (
    col, initcap, trim, lower, upper, regexp_replace, regexp_extract,
    when, to_date, lit, current_date, datediff, length, concat_ws
)

from datetime import datetime, timedelta

In [0]:
df_raw = spark.table(f'{catalog}.{bronze_db}.{trans_tbl}')
# DBTITLE 1


In [0]:
df_raw.count()

- Convert `Transaction_Date`, `DOB`, `Expiry_Date` to DATE
- Filter out records:
  - `DOB` must be in the past: `datediff(current_date(), DOB) > 0`
  - `Expiry_Date` must be in the future: `datediff(Expiry_Date, current_date()) > 0`

In [0]:
df = df_raw.withColumn("DOB", to_date(col("DOB"), "yyyy-MM-dd")) \
           .withColumn("Transaction_Date", to_date(col("Transaction_Date"), "yyyy-MM-dd")) \
           .withColumn("Expiry_Date", to_date(col("Expiry_Date"), "yyyy-MM-dd")) \
           .filter((datediff(current_date(), col("DOB")) > 0) &
                   (datediff(col("Expiry_Date"), current_date()) > 0))

- For fields like `Customer_Name`, `Merchant_Name`, `City`, `State`, `Country`, `Issuer_Bank`:
  - Remove non-alphabetic characters
  - Trim whitespace
  - Apply InitCap() (title case)
- Remove rows with missing values in the columns: `Transaction_ID`, `Customer_ID`, and `Card_ID`
- Remove duplicate rows based on the `Transaction_ID` column, keeping the first occurrence

In [0]:
df.display()

print("After applying filter, count is : ",df.count())


- For fields like `Customer_Name`, `Merchant_Name`, `City`, `State`, `Country`, `Issuer_Bank`:
  - Remove non-alphabetic characters
  - Trim whitespace
  - Apply InitCap() (title case)
- Remove rows with missing values in the columns: `Transaction_ID`, `Customer_ID`, and `Card_ID`
- Remove duplicate rows based on the `Transaction_ID` column, keeping the first occurrence

In [0]:
text_fields = [
    "Customer_Name", "Merchant_Name", "Customer_City", "City", "State",
    "Country", "Merchant_Country", "Issuer_Bank", "Card_Type"
]

for f in text_fields:
    df = df.withColumn(f,
            initcap(trim(regexp_replace(col(f), r"[^a-zA-Z\\s]", ""))))

df.display()
print(df.count())

> ####**Clean `Gender`:**  
> - Normalize variants (e.g., `"M"`, `"MALE"` → `"MALE"`; `"F"`, `"FEMALE"` → `"FEMALE"`)  
> - Trim whitespace

In [0]:
from pyspark.sql.functions import col, upper, trim, when, lit

df = df.withColumn(
    "Gender",
    when(upper(trim(col("Gender"))) == "M", upper(trim(lit("Male"))))
    .when(upper(trim(col("Gender"))) == "F", upper(trim(lit("Female"))))
    .otherwise(upper(trim(lit("Other"))))
)


- `dropna(subset=[...])`
  - Removes rows where any of the columns `Transaction_ID`, `Customer_ID`, or `Card_ID` have a missing (NaN) value.
  - Only these columns are checked; other columns may still contain NaNs.

- `drop_duplicates(["Transaction_ID"])`
  - After removing missing values, removes duplicate rows based on the `Transaction_ID` column.
  - If multiple rows share the same `Transaction_ID`, only the first occurrence is kept.

In [0]:
df = df.dropna(subset=["Transaction_ID", "Customer_ID", "Card_ID"]) \
           .drop_duplicates(["Transaction_ID"])

- Clean `Card_Tier`:
  - Trim whitespace
  - Convert to UPPERCASE

In [0]:
#Sample data from card_tier column:
df.select(col("Card_Tier")).show(5)

In [0]:
df = df.withColumn("Card_Tier", upper(trim(col("Card_Tier"))))

- Clean `Email`:
  - Regex match: `^[\w\.-]+@[\w\.-]+\.\w+$`
    - If valid → lowercased, trimmed
    - If invalid → set to NULL

In [0]:
df = df.withColumn(
        "Email",
        when(col("Email").rlike(r"^[\w\.-]+@[\w\.-]+\.\w+$"), lower(trim(col("Email"))))
        .otherwise(lit(None)))


- If `Customer_Name` is NULL and `Email` is valid:
  - Extract string before `@` and use as name
  - Else retain original value

In [0]:
print("Null count: ", df.select("Customer_Name").filter(col("Customer_Name").isNull()).count())

df.select("Customer_Name","Email").filter(col("Customer_Name").isNull()).show()


In [0]:
df = df.withColumn(
    "Customer_Name",
    when(col("Customer_Name").isNull(),
         regexp_extract(col("Email"), r"^([^@]+)", 1))
    .otherwise(col("Customer_Name"))
)

- Clean `Phone_Number`:
  - Keep only digits
  - Must be exactly 10 digits
    - If valid → retain
    - If invalid → set to NULL

In [0]:
df.filter(length(col("Phone")) > 10).display()


In [0]:
df = df.withColumn(
    "Phone",
    when((col("Phone").rlike(r"^\d{10}$")), col("Phone"))
    .otherwise(lit(None))
)


- Replace the following placeholder values with NULL:
  - “-”, “null”, “unknown”, “N/A”, “na”, “none”, “None”
- Applicable fields:
  - Customer_Name, Merchant_Name, City, State, Country, Bank, Card_Tier, Transaction_Status, Transaction_Type, Email, Phone

In [0]:
placeholders = ["-", "null", "unknown", "N/A", "na", "none", "None"]
string_cols = [
    "Customer_Name", "Merchant_Name", "Customer_City", "City", "State",
    "Country", "Merchant_Country", "Issuer_Bank", "Card_Type",
    "Transaction_Status", "Transaction_Type", "Email", "Phone"
]

for f in string_cols:
    df = df.withColumn(
            f,
            when(col(f).isin(placeholders), lit(None)).otherwise(col(f))
        )

- Correct known typos:
  - "Tmil Nadu" → "Tamil Nadu" in State
  - "Inda" → "India" in Country

In [0]:
df = (df
      .withColumn("State",   regexp_replace(col("State"),   "Tmil Nadu", "Tamil Nadu"))
      .withColumn("Country", regexp_replace(col("Country"), "Inda",      "India")))

- Add `Is_Valid_Record` column:
  - True if all conditions are met:
    - `Transaction_Amount` > 0
    - `DOB` < current_date()
    - `Expiry_Date` > current_date()
  - Otherwise, False

In [0]:


df = df.withColumn(
    "Is_Valid_Record",
    when(
        (col("Transaction_Amount") > 0) &
        (datediff(current_date(), col("DOB")) > 0) &             # DOB must be in the past
        (datediff(col("Expiry_Date"), current_date()) > 0),   # Expiry date must be in the future
        lit(True)
    ).otherwise(lit(False))
)


In [0]:
df.display()

- Add `Invalid_Reason` column with priority:
  - If `Transaction_Amount` <= 0 → "Invalid Transaction Amount"
  - Else if `DOB` >= current_date() → "Future DOB"
  - Else if `Expiry_Date` <= current_date() → "Card Expired"
  - Else NULL

In [0]:
# 2. Add Invalid_Reason
df = df.withColumn(
    "Invalid_Reason",
    when(col("Transaction_Amount") <= 0, lit("Invalid Transaction Amount"))
    .when(datediff(current_date(), col("DOB")) <= 0, lit("Future DOB"))
    .when(datediff(col("Expiry_Date"), current_date()) <= 0, lit("Card Expired"))
    .otherwise(lit(None))
)

city_state_map = `{
    "Bangalore"  : ("Karnataka",       "L1001"),
    "Mumbai"     : ("Maharashtra",     "L1002"),
    "Chennai"    : ("Tamil Nadu",      "L1003"),
    "Delhi"      : ("Delhi",           "L1004"),
    "Kolkata"    : ("West Bengal",     "L1005"),
    "Hyderabad"  : ("Telangana",       "L1006"),
    "Lucknow"    : ("Uttar Pradesh",   "L1007"),
    "Ahmedabad"  : ("Gujarat",         "L1008"),
    "Chandigarh" : ("Punjab",          "L1009"),
    "Kochi"      : ("Kerala",          "L1010"),
}`
- Normalize City and State to title case (trimmed)
- Use reference mapping (`city_state_map`) to:
  - Force correct State for known cities
  - Enrich with `Location_ID`
- Default `Location_ID` = 'L9999' if no match found

In [0]:
from pyspark.sql.functions import col, when, lit, trim, initcap

def apply_city_state(df, mapping):
    df_out = df
    for city, (state, loc) in mapping.items():
        df_out = df_out.withColumn(
            "State",
            when(col("City") == city, lit(state)).otherwise(col("State"))
        ).withColumn(
            "Location_ID",
            when((col("City") == city) & (col("State") == state), lit(loc)).otherwise(col("Location_ID"))
        )
    df_out = df_out.fillna({"Location_ID": "L9999"})
    return df_out

# Example usage and explanation
city_state_map = {
    "Bangalore"  : ("Karnataka",       "L1001"),
    "Mumbai"     : ("Maharashtra",     "L1002"),
    "Chennai"    : ("Tamil Nadu",      "L1003"),
    "Delhi"      : ("Delhi",           "L1004"),
    "Kolkata"    : ("West Bengal",     "L1005"),
    "Hyderabad"  : ("Telangana",       "L1006"),
    "Lucknow"    : ("Uttar Pradesh",   "L1007"),
    "Ahmedabad"  : ("Gujarat",         "L1008"),
    "Chandigarh" : ("Punjab",          "L1009"),
    "Kochi"      : ("Kerala",          "L1010"),
}

# Suppose df is already defined and cleaned as in previous cells
df_result = apply_city_state(df, city_state_map)

# Show the result to verify the function works as expected
# display(df_result)

# Explain: The function iterates over the mapping, updating 'State' and 'Location_ID' columns based on city matches.
# If a city matches, its state is set; if both city and state match, the location_id is set.
# Any missing location_id is filled with 'L9999'.

In [0]:
display(df_result)



- Normalize `Transaction_Status` values (case-insensitive):
  - "sucsess", "success" → **Success**
  - "fail", "failed" → **Failed**
  - "fraudulent" → **Fraudulent**
  - Others → **Unknown**

- Normalize `Transaction_Type` values (case-insensitive):
  - "atm" → **ATM**
  - "pos" → **POS**
  - "online", "onlne" → **ONLINE**
  - Others → **UNKNOWN**

In [0]:
df = df.withColumn(
    "Transaction_Status",
    when(lower(trim(col("Transaction_Status"))).isin("sucsess", "success"),    lit("Success"))
    .when(lower(trim(col("Transaction_Status"))).isin("fail",  "faild"  "failed"),    lit("Failed"))
    .when(lower(trim(col("Transaction_Status")))== "fraudulent",               lit("Fraudulent"))
    .otherwise(lit("Unknown"))
)

df = df.withColumn(
    "Transaction_Type",
    when(lower(trim(col("Transaction_Type"))) == "atm",     lit("ATM"))
    .when(lower(trim(col("Transaction_Type"))) == "pos",     lit("POS"))
    .when(lower(trim(col("Transaction_Type"))).isin("online","onlne"), lit("ONLINE"))
    .otherwise(lit("UNKNOWN"))
)

In [0]:
# 4. Column order as per target table
target_cols = [
    "Transaction_ID", "Customer_ID", "Card_ID", "Merchant_ID", "Location_ID",
    "Transaction_Date", "DOB", "Expiry_Date",
    "Customer_Name", "Gender", "Email", "Phone", "Customer_City",
    "Card_Type", "Issuer_Bank", "Card_Tier",
     "Transaction_Status", "Transaction_Type", "Transaction_Amount",
    "Merchant_Name", "Merchant_Category", "Merchant_Country",
    "City", "State", "Country"
]
df_final = df.select([col(c) for c in target_cols])

In [0]:
# Write to Silver Delta table (overwrite)
df_final.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{catalog}.{silver_db}.{cleaned_tbl}")

In [0]:
%sql
select * from dev.db1_silver.clean_transactions limit 10;